In [28]:
import pandas as pd
# Load the dataset
df = pd.read_excel("/Users/muyan/Desktop/SKU1_09_09.xlsx")

# 删除完全为空的列
df_cleaned = df.dropna(axis=1, how='all')
# 删除包含空值的行
df_cleaned = df_cleaned.dropna(axis=0, how='any')
#df_cleaned.head()

In [29]:
#1.以`salesdate`作为索引，星期，季度，是否节假日从`salesdate`中提取出来作为定类变量
# 设置salesdate为索引并将其转换为日期格式
df_cleaned['salesdate'] = pd.to_datetime(df_cleaned['salesdate'])
df_cleaned.set_index('salesdate', inplace=True)
# 从salesdate中提取星期、季度和节假日
holidays = ['2022-01-01', '2022-05-01', '2022-10-01']  # 定义节假日
df_cleaned['weekday'] = df_cleaned.index.dayofweek     # 提取星期
df_cleaned['quarter'] = df_cleaned.index.quarter       # 提取季度
df_cleaned['is_holiday'] = df_cleaned.index.isin(holidays).astype(int)  # 判断是否为节假日
df_cleaned[['weekday', 'quarter', 'is_holiday']].head()



In [23]:
#2.计算两个竞品与当天定价的差值与比率作为新的定量特征。
# 计算与comp_1_price的价格差和比率
df_cleaned['diff_price_comp_1'] = df_cleaned['comp_1_price'] - df_cleaned['price']
df_cleaned['ratio_price_comp_1'] = df_cleaned['comp_1_price'] / df_cleaned['price']

# 计算与comp_2_price的价格差和比率
df_cleaned['diff_price_comp_2'] = df_cleaned['comp_2_price'] - df_cleaned['price']
df_cleaned['ratio_price_comp_2'] = df_cleaned['comp_2_price'] / df_cleaned['price']

df_cleaned[['price', 'comp_1_price', 'diff_price_comp_1', 'ratio_price_comp_1', 'comp_2_price', 'diff_price_comp_2', 'ratio_price_comp_2']].head()

,price,comp_1_price,diff_price_comp_1,ratio_price_comp_1,comp_2_price,diff_price_comp_2,ratio_price_comp_2
salesdate,,,,,,,
2022-04-18,27.49,29.95,2.46,1.089487,29.99,2.5,1.090942
2022-04-19,27.79,29.95,2.16,1.077726,29.99,2.2,1.079165
2022-04-20,27.79,29.95,2.16,1.077726,29.99,2.2,1.079165
2022-04-21,27.69,29.95,2.26,1.081618,29.99,2.3,1.083062
2022-04-22,27.69,29.95,2.26,1.081618,29.99,2.3,1.083062


In [24]:
#3.定义一个新列表示简单观察仓储耗损情况，计算方法是用前一天的stock level减去当天的数值，并比上前一天的`unitsordered`
# 计算仓储耗损
df_cleaned['previous_stock'] = df_cleaned['managed_fba_stock_level'].shift(1)
df_cleaned['previous_unitsordered'] = df_cleaned['unitsordered'].shift(1)
df_cleaned['storage_loss'] = (df_cleaned['previous_stock'] - df_cleaned['managed_fba_stock_level']) / df_cleaned['previous_unitsordered']
df_cleaned[['managed_fba_stock_level', 'previous_stock', 'unitsordered', 'previous_unitsordered', 'storage_loss']].head()

,managed_fba_stock_level,previous_stock,unitsordered,previous_unitsordered,storage_loss
salesdate,,,,,
2022-04-18,2001.0,NaN,72,NaN,NaN
2022-04-19,2001.0,2001.0,105,72.0,0.000000
2022-04-20,2481.0,2001.0,92,105.0,-4.571429
2022-04-21,2433.0,2481.0,108,92.0,0.521739
2022-04-22,2337.0,2433.0,95,108.0,0.888889


In [25]:
#4.对前7个日期的 unitsordered 和 price 取平均
# 计算前7天的unitsordered和price的滚动平均值
df_cleaned['avg_unitsordered_7days'] = df_cleaned['unitsordered'].rolling(window=7).mean()
df_cleaned['avg_price_7days'] = df_cleaned['price'].rolling(window=7).mean()

df_cleaned[['unitsordered', 'avg_unitsordered_7days', 'price', 'avg_price_7days']].head()

,unitsordered,avg_unitsordered_7days,price,avg_price_7days
salesdate,,,,
2022-04-18,72,NaN,27.49,NaN
2022-04-19,105,NaN,27.79,NaN
2022-04-20,92,NaN,27.79,NaN
2022-04-21,108,NaN,27.69,NaN
2022-04-22,95,NaN,27.69,NaN
